In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.5/644.5 kB 10.2 MB/s eta 0:00:00


In [ ]:
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

In [ ]:
def train_test_split(path, neg_path=None, split = 0.2):
  print("----PROCESS STARTED------")


  files = list(set([name[:-4] for name in os.listdir(path)]))##removing duplicate names i.e. counting only number of images


  print(f"---This folder has a total number of {len(files)} images---")
  random.seed(42)
  random.shuffle(files)

  test_size = int(len(files)* split)
  train_size = len(files) - test_size
  ## creating required directories
  os.makedirs(train_path_img, exist_ok = True)
  os.makedirs(train_path_label, exist_ok = True)
  os.makedirs(val_path_img, exist_ok = True)
  os.makedirs(val_path_label, exist_ok = True)


  ##----copying image to train folder
  for filex in tqdm(files[:train_size]):
    if filex == 'classes':
      continue

    shutil.copy2(path + filex + '.jpg',f"{train_path_img}/"+ filex + '.jpg' )
    shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex +'.txt')


  print(f"-----training data created with 80% split {len(files[:test_size])} images ---")

  if neg_path:
    neg_images = list(set([name[:-4]for name in os.listdir(neg_path)])) ##remobing duplicate names i.e. counting only number of images
    for filex in tqdm(neg_images):
      shutil.copy2(neg_path+filex+".jpg" , f"{train_path_img}/" + filex + '.jpg')

    print(f"----Total {len(neg_images)}negative images added to the training data ----")





  ### copying images to validation folder
  for filex in tqdm(files[train_size:]):
    if filex == 'classes':
      continue

    # print ("running ")

    shutil.copy2(path + filex + '.jpg', f"{val_path_img}/"+ filex + '.jpg')
    shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

  print(f"---- Testing data created with a total of {len(files[train_size:])} images -----")

  print(f"----Task Completed ----")


train_test_split('/content/drive/MyDrive/YOLO/data/')


----PROCESS STARTED------
---This folder has a total number of 66 images---


  0%|          | 0/53 [00:00<?, ?it/s]

-----training data created with 80% split 13 images ---


  0%|          | 0/13 [00:00<?, ?it/s]

---- Testing data created with a total of 13 images -----
----Task Completed ----


In [ ]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.200 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.2/78.2 GB disk)


In [ ]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/YOLO/dataset.yaml epochs=10 imgsz=640 batch=4 project=/content/drive/MyDrive/YOLO/training_results name=basketball

100% 21.5M/21.5M [00:00<00:00, 294MB/s]
Ultralytics YOLOv8.0.200 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/YOLO/dataset.yaml, epochs=10, patience=50, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/YOLO/training_results, name=basketball2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agno

In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/YOLO/training_results/basketball/weights/best.pt conf = 0.35  source=/content/drive/MyDrive/YOLO/test_images

Ultralytics YOLOv8.0.200 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

image 1/4 /content/drive/MyDrive/YOLO/test_images/frame1.jpg: 384x640 9 players, 101.9ms
image 2/4 /content/drive/MyDrive/YOLO/test_images/frame2.jpg: 384x640 4 players, 11.1ms
image 3/4 /content/drive/MyDrive/YOLO/test_images/frame3.jpg: 384x640 8 players, 11.1ms
image 4/4 /content/drive/MyDrive/YOLO/test_images/frame4.jpg: 384x640 7 players, 11.1ms
Speed: 1.9ms preprocess, 33.8ms inference, 33.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/YOLO/output

In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/YOLO/training_results/basketball/weights/best.pt conf=0.5 source=/content/drive/MyDrive/YOLO/video

In [ ]:
!cp -r /content/runs/detect/predict5 /content/drive/MyDrive/YOLO/output